# Sentiment analysis using BiRNN

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

In [ ]:
dataset = load_dataset("behbudiy/uzbek-sentiment-analysis")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/673 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
train_size = int(0.8 * len(tokenized_datasets["train"]))  # 80% for training
test_size = len(tokenized_datasets["train"]) - train_size  # 20% for testing


In [ ]:
from torch.utils.data import random_split
train_dataset, test_dataset = random_split(tokenized_datasets["train"], [train_size, test_size])


In [ ]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, bidirectional=True)
        self.decoder = nn.Linear(4 * num_hiddens, 2)  # Binary classification

    def forward(self, inputs):
        embeddings = self.embedding(inputs.T)  # Transpose for LSTM
        self.encoder.flatten_parameters()
        outputs, _ = self.encoder(embeddings)  # Outputs shape: (time steps, batch size, 2*num_hiddens)
        encoding = torch.cat((outputs[0], outputs[-1]), dim=1)  # Concatenate first and last states
        outs = self.decoder(encoding)
        return outs

In [ ]:
vocab_size = tokenizer.vocab_size
embed_size, num_hiddens, num_layers = 100, 100, 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = BiRNN(vocab_size, embed_size, num_hiddens, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [ ]:
def train_model(model, train_loader, test_loader, num_epochs, criterion, optimizer):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            inputs = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)


            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

    evaluate_model(model, test_loader)


def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            inputs = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(inputs)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    print(f"Accuracy: {correct / total:.4f}")

In [ ]:
num_epochs = 5
train_model(model, train_dataloader, test_dataloader, num_epochs, criterion, optimizer)


Epoch 1/5, Loss: 0.4271
Epoch 2/5, Loss: 0.2329
Epoch 3/5, Loss: 0.1637
Epoch 4/5, Loss: 0.1183
Epoch 5/5, Loss: 0.0994
Accuracy: 0.9100


In [ ]:
def predict_sentiment(model, tokenizer, text):
    model.eval()
    tokens = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    inputs = tokens["input_ids"].to(device)  # No need to transpose here
    with torch.no_grad():
        outputs = model(inputs)  # Directly pass inputs
        label = torch.argmax(outputs, dim=1).item()  # Take argmax over class dimension
    return "positive" if label == 1 else "negative"


In [ ]:

print(predict_sentiment(model, tokenizer, "Oka yaxwi chqmapt"))


negative
